In [16]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import re
import pickle

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.linear_model import LogisticRegression

from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, f1_score


In [5]:
df=pd.read_csv('tweets.csv',encoding='latin')

In [7]:
df.columns = ['target', 'tweet_id', 'time', 'flag', 'user', 'text']

In [8]:
df.head()

,target,tweet_id,time,flag,user,text
0,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
1,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
2,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
3,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."
4,0,1467811372,Mon Apr 06 22:20:00 PDT 2009,NO_QUERY,joy_wolf,@Kwesidei not the whole crew


Lets create a dataframe cith the target column and the text

In [9]:
features ='text'
target = 'target'

df1 = df[[features, target]]

In [10]:
df1.head()

,text,target
0,is upset that he can't update his Facebook by ...,0
1,@Kenichan I dived many times for the ball. Man...,0
2,my whole body feels itchy and like its on fire,0
3,"@nationwideclass no, it's not behaving at all....",0
4,@Kwesidei not the whole crew,0


In [12]:
#no nulls
df1.isnull().sum()


text      0
target    0
dtype: int64

In [13]:
df1.info

<bound method DataFrame.info of                                                       text  target
0        is upset that he can't update his Facebook by ...       0
1        @Kenichan I dived many times for the ball. Man...       0
2          my whole body feels itchy and like its on fire        0
3        @nationwideclass no, it's not behaving at all....       0
4                            @Kwesidei not the whole crew        0
...                                                    ...     ...
1599994  Just woke up. Having no school is the best fee...       4
1599995  TheWDB.com - Very cool to hear old Walt interv...       4
1599996  Are you ready for your MoJo Makeover? Ask me f...       4
1599997  Happy 38th Birthday to my boo of alll time!!! ...       4
1599998  happy #charitytuesday @theNSPCC @SparksCharity...       4

[1599999 rows x 2 columns]>

In [14]:
df1['target'].value_counts()

4    800000
0    799999
Name: target, dtype: int64

In [17]:
# take out the @usernames
df1['text'] = df1['text'].apply(lambda x: re.sub('@[^\s]+','',x))

# take out the links
df1["text"] = df1["text"].apply(lambda x: re.sub(r"http\S+", "", x))

# take out the hashtags
df1["text"] = df1["text"].apply(lambda x: re.sub(r"#\S+", "", x))

C:\Users\janem\AppData\Local\Temp\ipykernel_9600\2827898166.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1['text'] = df1['text'].apply(lambda x: re.sub('@[^\s]+','',x))
C:\Users\janem\AppData\Local\Temp\ipykernel_9600\2827898166.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1["text"] = df1["text"].apply(lambda x: re.sub(r"http\S+", "", x))
C:\Users\janem\AppData\Local\Temp\ipykernel_9600\2827898166.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFr

In [19]:
df1.head()

,text,target
0,is upset that he can't update his Facebook by ...,0
1,I dived many times for the ball. Managed to s...,0
2,my whole body feels itchy and like its on fire,0
3,"no, it's not behaving at all. i'm mad. why am...",0
4,not the whole crew,0


In [20]:
X_train, X_test, y_train, y_test = train_test_split(df["text"], df["target"], test_size=0.3, random_state=102)

In [21]:
vectorizer = TfidfVectorizer()
X_train = vectorizer.fit_transform(X_train)
X_test = vectorizer.transform(X_test)

In [22]:
lr = LogisticRegression()
lr.fit(X_train, y_train)
lr.score(X_test, y_test)

C:\Users\janem\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


0.8024791666666666

In [24]:
predict = lambda x: lr.predict(vectorizer.transform([x]))
sentiment  = lambda x: 'positive' if predict(x) == 4 else 'negative'

In [30]:
#thus you can predict either of the tweets as either negative or positive
tweet = "not the whole crew"
sentiment(tweet)

'negative'

In [28]:
df1.head()

,text,target
0,is upset that he can't update his Facebook by ...,0
1,I dived many times for the ball. Managed to s...,0
2,my whole body feels itchy and like its on fire,0
3,"no, it's not behaving at all. i'm mad. why am...",0
4,not the whole crew,0
